In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import cv2


In [2]:


loadedImages = []
for dirname, _, filenames in os.walk("D:\\HandGesture\\images\\classone"):
    for i in range(0, 600):
        image = cv2.imread(os.path.join(dirname, filenames[i]))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.resize(gray_image, dsize=(28,28), interpolation=cv2.INTER_CUBIC)
        loadedImages.append(gray_image.reshape(28*28))
for dirname, _, filenames in os.walk("D:\\HandGesture\\images\\classtwo"):
    for i in range(0, 460):
        image = cv2.imread(os.path.join(dirname, filenames[i]))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.resize(gray_image, dsize=(28,28), interpolation=cv2.INTER_CUBIC)
        loadedImages.append(gray_image.reshape(28*28))
for dirname, _, filenames in os.walk("D:\\HandGesture\\images\\classthree"):
    for i in range(0, 700):
        image = cv2.imread(os.path.join(dirname, filenames[i]))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.resize(gray_image, dsize=(28,28), interpolation=cv2.INTER_CUBIC)
        loadedImages.append(gray_image.reshape(28*28))

In [3]:
outputVectors = []
for i in range(0, 600):
    outputVectors.append([1, 0, 0])

for i in range(0, 460):
    outputVectors.append([0, 1, 0])

for i in range(0, 700):
    outputVectors.append([0, 0, 1])

In [4]:
import random
cc = list(zip(loadedImages, outputVectors))
random.shuffle(cc)
loadedImages[:], outputVectors[:] = zip(*cc)

loadedImages = np.array(loadedImages)
outputVectors = np.array(outputVectors)
X_dev = loadedImages[0:60].T
Y_dev = outputVectors[0:60].T
X_dev =  (X_dev+40) / 295.

X_train = loadedImages[60:1760].T
Y_train = outputVectors[60:1760].T
X_train = (X_train + 40)/295.
_,m_train = X_train.shape

In [5]:
m = m_train
#初始化权重参数
def init_params():
    W1 = np.random.rand(100, 784)
    b1 = np.random.rand(100, 1) 
    W2 = np.random.rand(3, 100)
    b2 = np.random.rand(3, 1)
    return W1, b1, W2, b2
#激活函数Relu，虽然它很普通，但是它反向传播很简单，即使它有可能让神经元死亡
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
#激活函数加归一化，上面之所以注释掉是因为如果不加处理，会容易梯度爆炸或者梯度消失
def softmax(x):
    max = np.max(x)
    return np.exp(x-max)/sum(np.exp(x-max))
#前向过程
def forward_prop(W1, b1, W2, b2 ,X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2
#反向传播过程中Relu的导数
def ReLU_deriv(Z):
    return Z > 0
#分类问题常用 独热编码
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = Y
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2,  dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2  
    return W1, b1, W2, b2

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2= init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2= forward_prop(W1, b1, W2, b2 ,X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2 ,W1, W2, X, Y)
        W1, b1, W2, b2  = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            Y1 = get_predictions(Y)
            print(get_accuracy(predictions, Y1))
    return W1, b1, W2, b2

In [7]:
# W1, b1, W2, b2= gradient_descent(X_train, Y_train,0.1, 50000)
# W1, b1, W2, b2= gradient_descent(X_train, Y_train,0.05, 30000,X_dev,Y_dev)
# #W1, b1, W2, b2= gradient_descent(X_train, Y_train,0.01, 60000)

# import os
# os.chdir(r"C:\\Users\\ww\\Desktop\\weight")
# np.save('W11.npy', W1)
# np.save('W22.npy', W2)
# np.save('b11.npy', b1)
# np.save('b22.npy', b2)
#


In [8]:
import os

os.chdir(r"C:\\Users\\ww\\Desktop\\weight")
i = 1300
#加载权重
W1 = np.load('W11.npy')
W2 = np.load('W22.npy')
b1 = np.load('b11.npy')
b2 = np.load('b22.npy')

In [9]:

import cv2
import numpy as np
import time
import math
# 打开摄像头
cap = cv2.VideoCapture(0)
# 手势肤色的颜色区间
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)
classone_image = cv2.imread("D:\\HandGesture\\images\\classone\\0.jpg")
classtwo_image = cv2.imread("D:\\HandGesture\\images\\classtwo\\0.jpg")
classthree_image = cv2.imread("D:\\HandGesture\\images\\classthree\\0.jpg")

while True:
    # 读取每一帧
    ret, frame = cap.read()
    # 镜像翻转图像
    frame = cv2.flip(frame, 1)
    kernel = np.ones((3,3),np.uint8)

    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    # 提取皮肤所在的区域
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    # 将皮肤所在的区域填充完整
    mask = cv2.dilate(mask,kernel,iterations = 4)
    # 将边缘润滑
    mask = cv2.GaussianBlur(mask,(5,5),100)
    frame[mask==0] = (0,0,0)
    frame[mask!=0] = (255,255,255)
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(gray_image, dsize=(28,28), interpolation=cv2.INTER_CUBIC).reshape(28*28,1)
    Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, (gray_image+40)/295)
    predictions = get_predictions(A2)
    print(predictions)
    cv2.imshow('Detection1', frame)
    if predictions ==0:
        cv2.imshow('label_net', classone_image)
    elif predictions ==1:
        cv2.imshow('label_net', classtwo_image)
    else :
        cv2.imshow('label_net', classthree_image)
    #用来采集图像的代码
    if cv2.waitKey(1) & 0xFF == ord('q'):
        #cv2.imwrite(r"E:\Test\\"+ str(i) + ".jpg",frame)
        cv2.imwrite(r"D:\\HandGesture\\images\\classtwo\\"+ str(i) + ".jpg",frame)
        i = i + 1
    #find contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    try:
        #find contour of max area(hand)
        cnt = max(contours, key=cv2.contourArea)
        #epsilon = 0.0005*cv2.arcLength(cnt,True)
        #approx= cv2.approxPolyDP(cnt,epsilon,True)
        #make convex hull around hand
        hull = cv2.convexHull(cnt,returnPoints = False)
        defects = cv2.convexityDefects(cnt,hull)
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])
            cv2.line(frame,start,end,[0,255,0],2)
            cv2.circle(frame,far,5,[0,0,255],-1)
        convex = defects
        count = 0
        for i in range(convex.shape[0]):
            s,e,f,d = convex[i][0]
            start = cnt[s][0]
            end = cnt[e][0]
            far = cnt[f][0]
            #计算三角形的三边的长度
            a = math.sqrt((start[0]-far[0])**2+(start[1]-far[1])**2)
            b = math.sqrt((end[0]-far[0])**2+(end[1]-far[1])**2)
            c = math.sqrt((start[0]-end[0])**2+(start[1]-end[1])**2)
            #计算角度
            angle = math.acos((a**2+b**2-c**2)/(2*a*b))*57
            #画出角度

            # #在角上放上角的度数
            # cv2.putText(finger_img,str(angle),far,cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),1)
            #判断角度小于九十度的角的个数
            if angle<90:
                count += 1
        
        if count <=1:
            cv2.imshow('label_final', classone_image)
        elif count >=4:
            cv2.imshow('label_final', classthree_image)
        else:
            cv2.imshow('label_final', classtwo_image)

        # #define area of hull and area of hand
        # areahull = cv2.contourArea(hull)
        # areacnt = cv2.contourArea(cnt)
        # #find the percentage of area not covered by hand in convex hull
        # arearatio=((areahull-areacnt)/areacnt)*100
        # #mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
        # cv2.polylines(frame, [cnt], True, (0, 255, 0), 3)
        # cv2.polylines(frame, [hull], True, (0, 255, 0), 3)
        cv2.imshow('Detection', frame)
    except:
        print('ture')
    #if cv2.waitKey(1) == 27:
    
    if cv2.waitKey(5) == 27:
        break

#释放摄像头，关闭窗口
cap.release()
cv2.destroyAllWindows()

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[2]
[2]
[2]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[2]
[2]
[2]
[2]
[2]
[2]
[0]
[0]
[2]
[2]
[2]
[2]
[0]
[0]
[0]
[0]
[2]
[1]
[0]
[0]
[0]
[1]
[2]
[2]
[0]
[0]
[0]
[2]
[2]
[2]
[1]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[0]
[0]
[2]
[0]
[0]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[0]
[0]
[2]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[0]
[0]
[0]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[1]
[2]
[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[2]
[2]
[0]
[0]
[0]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[0]
[0]
[0]
[2]
[2]
[2]
[0]
